In [3]:
import time
import numpy as np
import pandas as pd
from utils import mysqlDatabase, dcardApi

In [4]:
class commentCrawler:
    def __init__(self, database_username, database_password, database_ip, database_name, base_url, popular, max_limit):
        self.database_username = database_username
        self.database_password = database_password
        self.database_ip = database_ip
        self.database_name = database_name
        self.base_url = base_url
        self.popular = popular
        self.max_limit = max_limit
    def get_postid_list(self, MysqlDatabase, sql):
        df = MysqlDatabase.select_table(sql)
        postid_list = df['postid'].tolist()
        return postid_list
    def get_comment(self, DcardApi, postid):
        df = DcardApi.get_Dcard_posts_comments(postid)
        return df
    def main(self):
        '''
        實作
        '''
        MysqlDatabase = mysqlDatabase(self.database_username, self.database_password, self.database_ip, self.database_name)
        DcardApi = dcardApi(self.base_url, self.popular, self.max_limit)
        ## 文章列表
        sql = '''SELECT distinct dp.postid, dp.title 
        FROM Bigdata.dcard_posts dp
        where dp.commentCount >= 30
        and not exists(
            select id from Bigdata.dcard_comments dc where dc.id = dp.postid
        )
        '''
        postid_list = self.get_postid_list(MysqlDatabase, sql)
        print('length of postid_list: ', len(postid_list))
        ## 打api拿留言
        df_comment = pd.DataFrame(columns=['id', 'anonymous', 'postId', 'createdAt', 'floor', 'content', 'likeCount', 'withNickname', 'gender', 'school', 'department'])
        time_start = time.time() 
        for postid in postid_list:
            time.sleep(1)
            time_cost = time.time() - time_start
            if time_cost >= 3600:
                break
            try:
                print(postid)
                df_comment = pd.concat([df_comment, self.get_comment(DcardApi, postid)])
            except:
                print('connection abort, need some break')
                time.sleep(5)
                continue
        print('total time cost(sec): ', time_cost)
        df_comment = df_comment.fillna('')[['id', 'anonymous', 'postId', 'createdAt', 'floor', 'content', 'likeCount', 'withNickname', 'gender', 'school', 'department']]
        MysqlDatabase.insert_table(df_comment, table_name='dcard_comments')
        return df_comment

In [5]:
if __name__ == '__main__':
    database_username = 'jackyfu'
    database_password = 'data8756'
    database_ip       = 'sg2nlmysql29plsk.secureserver.net'
    database_name     = 'Bigdata'
    base_url = 'https://www.dcard.tw/service/api/v2'
    popular = 'false'
    max_limit = '100'
    ###
    CommentCrawler = commentCrawler(database_username, database_password, database_ip, database_name, base_url, popular, max_limit)
    df_comment = CommentCrawler.main()

Successfully select from Bigdata table
length of postid_list:  364
236808962
236807724
236682003
236806608
236805307
236803687
connection abort, need some break
236806003
236808552
236809513
236810114
236806873
236808339
236805438
236811796
236806384
236807624
236809533
236804662
236804236
236806522
236809074
connection abort, need some break
236806805
236807847
236804628
236810018
236812367
236806184
236804584
236809648
236810251
236807801
connection abort, need some break
236809544
connection abort, need some break
236811779
connection abort, need some break
236768502
connection abort, need some break
236804373
connection abort, need some break
236806177
connection abort, need some break
236806398
connection abort, need some break
236807669
connection abort, need some break
236810998
connection abort, need some break
236813642
connection abort, need some break
236808999
connection abort, need some break
236805080
connection abort, need some break
236805821
connection abort, need some

In [6]:
df_comment

,id,anonymous,postId,createdAt,floor,content,likeCount,withNickname,gender,school,department
0,afec17bd-b835-46f8-a5db-437b021a1c27,True,236808962,2021-08-25T17:28:25.019Z,1,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,21,False,M,長庚大學,
1,cab1968a-c5db-40b3-be64-eff72e9d1e09,False,236808962,2021-08-25T17:28:48.662Z,2,騷擾你也是我的自由 ...,10,True,M,亞洲董神,larry
2,2f20fc10-ca1b-4dca-9bef-dee0ad86054d,False,236808962,2021-08-25T17:28:55.721Z,3,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,14,True,M,拿鐵大學 不想上班所,montag
3,e2bec33b-c995-4941-a13b-2b8f68264de6,False,236808962,2021-08-25T17:29:02.788Z,4,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,8,True,M,亞洲董神,larry
4,93cbb45b-0be8-4809-b953-c987b376261b,False,236808962,2021-08-25T17:30:15.348Z,5,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,9,True,M,Eligos,devil_egg
...,...,...,...,...,...,...,...,...,...,...,...
25,5d53f3a0-8ffe-4aae-957e-e34cd548d517,False,236805546,2021-08-26T04:56:43.848Z,26,B17 已經研究股票很久了，這篇文也根本沒看網路教學啦～純粹想要分享籌碼資訊跟工具給大家～但...,0.0,True,F,桑尼ㄉ投資日記🌿,sunnylee0226
26,c3a6adf4-420e-4438-8856-33f3cb01b421,False,236805546,2021-08-26T05:02:53.081Z,27,,,True,,,
27,23266340-0548-4e2c-89f1-13d75d17124d,False,236805546,2021-08-26T05:07:44.756Z,28,如果股價不配合呢？？,0.0,True,M,資本主義下的肉便器,jesse.y
28,f67c76ad-d2bb-4b2c-b926-63f1cd19a829,True,236805546,2021-08-26T05:16:45.538Z,29,B28 跑，不要當洗碗那個,1.0,False,M,中國科技大學,


In [7]:
2610/30

87.0